In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose
from keras.models import Model
from keras.preprocessing import image

In [ ]:
cat_train_path = "../input/cat-and-dog/training_set/training_set/cats/" 
cat_test_path = "../input/cat-and-dog/test_set/test_set/cats/"

cat_train = []
for filename in os.listdir(cat_train_path):
    if filename.endswith(".jpg"):
        img = image.load_img(cat_train_path+filename, target_size=(128, 128))
        cat_train.append(image.img_to_array(img))
cat_train = np.array(cat_train)

cat_test = []
for filename in os.listdir(cat_test_path):
    if filename.endswith(".jpg"):
        img = image.load_img(cat_test_path+filename, target_size=(128, 128))
        cat_test.append(image.img_to_array(img))
cat_test = np.array(cat_test)

In [ ]:
print("cat_train", cat_train.shape)
print("cat_test", cat_test.shape)

In [ ]:
def show_cat_data(X, n=10, title=""):
    plt.figure(figsize=(15, 5))
    for i in range(n):
        ax = plt.subplot(2,n,i+1)
        plt.imshow(image.array_to_img(X[i]))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.suptitle(title, fontsize = 20)

In [ ]:
def noisy(image):
  row,col,ch= image.shape
  mean = 50
  var = 1024
  sigma = var**0.5
  gauss = np.random.normal(mean,sigma,(row,col,ch))
  gauss = gauss.reshape(row,col,ch)
  noisy = image + gauss
  return noisy

In [ ]:
cat_train_noisy = []
for img in cat_train:
    noisy_img = noisy(img)
    cat_train_noisy.append(noisy_img)
cat_train_noisy = np.array(cat_train_noisy)
print(cat_train_noisy.shape)

In [ ]:
cat_test_noisy = []
for img in cat_test:
    noisy_img = noisy(img)
    cat_test_noisy.append(noisy_img)
cat_test_noisy = np.array(cat_test_noisy)
print(cat_test_noisy.shape)

In [ ]:
show_cat_data(cat_train, title="train cats")
show_cat_data(cat_test, title="test cats")

In [ ]:
show_cat_data(cat_train, title="train cats")
show_cat_data(cat_train_noisy, title="train cats noisy")

In [ ]:
show_cat_data(cat_test, title="test cats")
show_cat_data(cat_test_noisy, title="test cats noisy")

In [ ]:
input_layer = Input(shape=(128, 128, 3), name="INPUT")
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_layer)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)

code_layer = MaxPooling2D((2, 2), name="CODE")(x)

x = Conv2DTranspose(8, (3, 3), activation='relu', padding='same')(code_layer)
x = UpSampling2D((2, 2))(x)
x = Conv2DTranspose(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2DTranspose(16, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2,2))(x)
output_layer = Conv2D(3, (3, 3), padding='same', name="OUTPUT")(x)

In [ ]:
optimizer = keras.optimizers.Adam(lr=0.01)

In [ ]:
cat_AE = Model(input_layer, output_layer)
cat_AE.compile(optimizer=optimizer, loss='mse',metrics="accuracy")
cat_AE.summary()

In [ ]:
history=cat_AE.fit(cat_train, cat_train,
                epochs=30,
                batch_size=32,
                shuffle=True,
                validation_data=(cat_test, cat_test))


In [ ]:
cat_AE.save("cat_AE.h5")

In [ ]:
get_encoded_cat = Model(inputs=cat_AE.input, outputs=cat_AE.get_layer("CODE").output)

In [ ]:
encoded_cat = get_encoded_cat.predict(cat_test)
encoded_cat = encoded_cat.reshape((len(cat_test), 16*16*8))
encoded_cat.shape

In [ ]:
reconstructed_cats = cat_AE.predict(cat_test_noisy)

In [ ]:
print("Test Accuracy: {:.2f}%".format(cat_AE.evaluate(np.array(cat_test_noisy),np.array(cat_test))[1]*100))

In [ ]:
show_cat_data(cat_test_noisy, title="original cats noisy")
show_cat_data(reconstructed_cats, title="reconstructed cats")

In [ ]:
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.legend()

In [ ]:
plt.plot(history.history["accuracy"], label="Training Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.legend()